In [ ]:
from mp.irregularPoly import irregularPoly
from mp.irregularPoly import generate_poly
from mp.robot import robot
import random
import utils.vox_mesh as vm
import copy
from run import obj_func
import pickle
import utils.sim_art as simart


# # component layer parameters
# irrPolyInd = {
#     "num_components": 3,
#     "evo_type": "component",
#     "grammar_iterations": 3,
#     "num_points": 10
# }

# # robot layer parameters
# robotInd = {
#     "evo_type": "robot",
#     "simulation_time": 3600,
#     "leg_lim": [1,3],
#     "contrl_trials": 1,
#     "contrl_trial_gen": 20,
#     "contrl_env_friction": 0.9,
#     "contrl_env_restitution": 1.0,
#     "contrl_env_xlim": [0, 10],
#     "contrl_env_ylim": [-5, 5],
#     "joint_ulim": 0.2,
#     "joint_llim": -0.2,
#     "contrl_delta_angle": 0.05,
#     "file_path": "data//",
# }


# def generate_leg(leg, archive):
#     square = [[0.11, 0.9], [0.11, 0.11], [0.9, 0.11], [0.9, 0.9]]
#     for i in range(0, leg.num_components):
#         new_poly, new_edge_in, new_edge_out = generate_poly(leg.dim_x, square)
#         leg.poly.append(new_poly)
#         leg.edge_idx_in.append(new_edge_in)
#         leg.edge_idx_out.append(new_edge_out)
#         leg.poly_materials.append(copy.deepcopy(archive[10]))
#     print(archive)
#     leg.id = vm.id_generator()


# def generate_robot(robo, archive):
#     robo.num_components = 3
#     # pick components from archive
#     for i in range(0, robo.num_components):
#         idx = random.randint(0, len(archive) - 1)
#         robo.parts.append(copy.deepcopy(archive[idx]))
#         robo.joints.append([0, 1, 0, 1])
#     robo.id = vm.id_generator()


# def evaluate():
#     random.seed(0)
#     friction_values = [0.25, 0.50, 0.75, 1.0]
#     restitution_values = [0.25, 0.50, 0.75, 1.0]

#     # friction_values = [1.0, 1.0, 1.0, 1.0]
#     # restitution_values = [1.0, 1.0, 1.0, 1.0]

#     materials = [[i, j] for i in friction_values
#                  for j in restitution_values]

#     # generate square legs
#     leg = irregularPoly(irrPolyInd)
#     generate_leg(leg, materials)
#     leg.grammar = [[0, 0], [0], [1]]
#     leg.axiom = 1
#     obj_func(leg)

#     # generate robot
#     robo = robot(robotInd)
#     generate_robot(robo, [leg])
#     obj_func(robo)
#     robo.fitness, pos = simart.simulate(robo, 2) 

#     print(robo.fitness)
#     print(pos)
#     print("awe")


#     alt_robo = copy.deepcopy(robo)
#     alt_robo.ctrl_env_friction = 0.05
#     alt_robo.ctrl_env_restitution = 0.05
#     alt_robo.fitness, pos = simart.simulate(alt_robo, 2)

#     # save robot
#     with open("data//"+robo.id + "_HFfit" + str(robo.fitness), 'wb') as pickle_file:
#         pickle.dump(robo, pickle_file)

#     with open("data//"+alt_robo.id + "_LFfit" + str(alt_robo.fitness), 'wb') as pickle_file:
#         pickle.dump(alt_robo, pickle_file)
#     print(alt_robo.fitness)
#     print(pos)

# if __name__ == "__main__":
#     evaluate()


In [ ]:
#A1ZQH
#186
import numpy as np
import random
import os
import statistics
from mp.map import map_elites
from mp.robot import robot
from mp.irregularPoly import irregularPoly
import utils.sim_art as simart
import utils.vox_mesh as vm
import pickle
import math
import multiprocessing
import shutil


# generations = [1000, 3000]
# niches = 1000
# exp_name = "sim3600moregen"

# run_folder = './' + exp_name + "_gen_" + str(1000) + "_" + str(generations[1]) + "_niche_" + str(niches) + "_"
# k=1000
# seed = 186
# robid = "A1ZQH"


# pickle_in = open(run_folder+"/toContinueLowerLayer.pickle","rb")
# archive = pickle.load(pickle_in)

# file2 = open(run_folder+"/LeftOffGeneration.txt","r")
# g = int(file2.readline())+1
# file2.close()

# layer_info = {"layer": 'layer3', "lower_layer": archive, "ind_params": robotInd}
# mp = map_elites(3, niches, 1000, default_params, layer_info)

# pickle_in = open(run_folder+"/toContinueUpperLayer.pickle","rb")
# archiveupper = pickle.load(pickle_in)
# mp.add_archive(archiveupper)
# #archive = mp.compute(obj_func, robot,g)
# layer_data_path = run_folder + "//run_data//layer3"

# #A1ZQH 3.577887662634793 0.49074074074074076 0.7037037037037037 0.5686527244911512



In [ ]:
default_params = \
    {
        # more of this -> higher-quality CVT
        "cvt_samples": 25000,
        # we evaluate in batches to parallelize
        "batch_size": 1,
        # proportion of niches to be filled before starting
        "random_init": 0.1,
        # batch for random initialization
        "random_init_batch": 1000,
        # when to write results (one generation = one batch)
        "dump_period": 10,
        # do we use several cores? ---we set this individually for different layers later in the code
        "parallel": False,
        # do we cache the result of CVT and reuse?
        "cvt_use_cache": True,
        # use xover and mutation or just mutation
        "mutation_only": True,
        # experiment folder
        "experiment_name": "sim3600moregen"
    }

robotInd = {
    "evo_type": "robot",
    "simulation_time": 1000000,
    "leg_lim": [1,3],
    "contrl_trials": 1,
    "contrl_trial_gen": 20,
    "contrl_env_friction": 0.9,
    "contrl_env_restitution": 1.0,
    "contrl_env_xlim": [0, 10],
    "contrl_env_ylim": [-5, 5],
    "joint_ulim": 0.2,
    "joint_llim": -0.2,
    "contrl_delta_angle": 0.05,
    "file_path": "",
}

friction_values = [0.25, 0.50, 0.75, 1.0]
restitution_values = [0.25, 0.50, 0.75, 1.0]

materials = [[i, j] for i in friction_values
                for j in restitution_values]

gen = [1000, 3000]
niches=1000

folder_name = default_params['experiment_name'] + "_gen_"+ str(gen[0]) + "_" + str(gen[1])+"_niche_" + str(niches)
root = folder_name

run_id = 28507
default_params['experiment_runid'] = run_id
run_folder = root + "_run" + str(run_id)
default_params['run_folder'] = run_folder
temp_solution_path = "data/"+run_folder+"//temp_solutions"
robotInd['file_path'] = temp_solution_path+"//temp"
try:
    shutil.rmtree(temp_solution_path)
except:
    pass
os.makedirs(temp_solution_path)


pickle_in = open(run_folder+"/toContinueLowerLayer.pickle","rb")
archive = pickle.load(pickle_in)

file2 = open(run_folder+"/LeftOffGeneration.txt","r")
g = int(file2.readline())+1
file2.close()

layer_info = {"layer": 'layer3', "lower_layer": archive, "ind_params": robotInd}
mp = map_elites(3, niches, 1000, default_params, layer_info)

pickle_in = open(run_folder+"/toContinueUpperLayer.pickle","rb")
archiveupper = pickle.load(pickle_in)

newarchive = {}
max = 0
for keys in archiveupper:
    if archiveupper[keys].fitness > max:
        newarchive = {}
        newarchive[keys] = archiveupper[keys]
        max = archiveupper[keys].fitness
        print(max)

mp.add_archive(newarchive)


archive = mp.compute(obj_func, robot,g)



layer_data_path = run_folder + "//run_data//layer3"


#save_archive(archive, layer_data_path)

#re_evaluate(archive, alternate_env[0], alternate_env[1], run_folder)
